In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

CONFERENCES = {
    "ACC": 2, "Big Ten": 7, "SEC": 8, "Big 12": 4, "Pac-12": 21, "Big East": 3, 
    "AAC": 62, "Mountain West": 46, "Atlantic 10": 3, "WCC": 26, "Sun Belt": 27,
    "MAAC": 5, "Horizon": 24, "Ivy League": 12, "Patriot League": 31, "America East": 1,
    "MEAC": 16, "SWAC": 23, "ASUN": 55, "Missouri Valley": 9, "CAA": 10,
    "Ohio Valley": 14, "Summit League": 44
}

def fetch_conference_fixtures(start_date, end_date):
    url_base = "https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard"
    all_games = []

    date_cursor = start_date
    while date_cursor <= end_date:
        date_str = date_cursor.strftime("%Y%m%d")
        print(f"📅 Fetching: {date_str}")

        for conf_name, conf_id in CONFERENCES.items():
            url = f"{url_base}?dates={date_str}&groups={conf_id}"
            resp = requests.get(url)

            if resp.status_code != 200:
                continue

            data = resp.json()
            for event in data.get("events", []):
                game = event["competitions"][0]
                teams = game["competitors"]

                all_games.append({
                    "Date": date_cursor.strftime("%d-%m-%Y"),
                    "Conference": conf_name,
                    "Home Team": teams[0]["team"]["displayName"],
                    "Away Team": teams[1]["team"]["displayName"],
                    "Home Score": teams[0].get("score", None),
                    "Away Score": teams[1].get("score", None),
                    "Status": game["status"]["type"]["description"]
                })

        date_cursor += timedelta(days=1)

    return pd.DataFrame(all_games)


# Run full season: Nov 1 to April 15
df = fetch_conference_fixtures(
    start_date=datetime(2025, 11, 24),
    end_date=datetime(2025, 12, 2)
)

df.to_excel("NCAA_Season_All_Conferences.xlsx", index=False)
print("\n✔ Completed. Total games scraped:", len(df))
print(df.head())


📅 Fetching: 20251124
📅 Fetching: 20251125
📅 Fetching: 20251126
📅 Fetching: 20251127
📅 Fetching: 20251128
📅 Fetching: 20251129
📅 Fetching: 20251130
📅 Fetching: 20251201
📅 Fetching: 20251202

✔ Completed. Total games scraped: 535
         Date Conference             Home Team                  Away Team  \
0  24-11-2025        ACC     NC State Wolfpack         Seton Hall Pirates   
1  24-11-2025        ACC       Kansas Jayhawks  Notre Dame Fighting Irish   
2  24-11-2025        ACC       Houston Cougars            Syracuse Orange   
3  24-11-2025        ACC  Louisville Cardinals    Eastern Michigan Eagles   
4  24-11-2025        ACC          SMU Mustangs        Radford Highlanders   

  Home Score Away Score     Status  
0          0          0  Scheduled  
1          0          0  Scheduled  
2          0          0  Scheduled  
3          0          0  Scheduled  
4          0          0  Scheduled  


In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

CONFERENCES = {
    "ACC": 2, "Big Ten": 7, "SEC": 8, "Big 12": 4, "Pac-12": 21, "Big East": 3, 
    "AAC": 62, "Mountain West": 46, "Atlantic 10": 3, "WCC": 26, "Sun Belt": 27,
    "MAAC": 5, "Horizon": 24, "Ivy League": 12, "Patriot League": 31, "America East": 1,
    "MEAC": 16, "SWAC": 23, "ASUN": 55, "Missouri Valley": 9, "CAA": 10,
    "Ohio Valley": 14, "Summit League": 44
}

def fetch_conference_fixtures(start_date, end_date):
    url_base = "https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard"
    all_games = []

    date_cursor = start_date
    while date_cursor <= end_date:
        date_str = date_cursor.strftime("%Y%m%d")
        print(f"📅 Fetching: {date_str}")

        for conf_name, conf_id in CONFERENCES.items():
            url = f"{url_base}?dates={date_str}&groups={conf_id}"
            resp = requests.get(url)

            if resp.status_code != 200:
                continue

            data = resp.json()
            for event in data.get("events", []):
                game = event["competitions"][0]
                teams = game["competitors"]

                all_games.append({
                    "Date": date_cursor.strftime("%d-%m-%Y"),
                    "Conference": conf_name,
                    "Home Team": teams[0]["team"]["displayName"],
                    "Away Team": teams[1]["team"]["displayName"],
                    "Home Score": teams[0].get("score", None),
                    "Away Score": teams[1].get("score", None),
                    "Status": game["status"]["type"]["description"]
                })

        date_cursor += timedelta(days=1)

    return pd.DataFrame(all_games)


# -------- RUN SCRAPER --------
df = fetch_conference_fixtures(
    start_date=datetime(2025, 11, 25),
    end_date=datetime(2025, 11, 25)
)

# -------- REMOVE DUPLICATES --------

# Normalize text (fix spaces/cases)
df["Home Team"] = df["Home Team"].str.upper().str.strip()
df["Away Team"] = df["Away Team"].str.upper().str.strip()

# Create unique match identifier
df["Game_Key"] = df["Date"] + "_" + df["Home Team"] + "_" + df["Away Team"]

# Extract duplicates for QA
duplicates_df = df[df.duplicated(subset=["Game_Key"], keep=False)].sort_values("Game_Key")
duplicates_df.to_excel("Duplicate_Fixtures.xlsx", index=False)

# Remove duplicates (keep only first entry)
clean_df = df.drop_duplicates(subset=["Game_Key"], keep="first")

# -------- SAVE OUTPUT --------
df.to_excel("RAW_NCAA_All_Games.xlsx", index=False)
clean_df.to_excel("CLEAN_NCAA_Fixtures.xlsx", index=False)

print("\n🔍 Total Raw Records:", len(df))
print("♻ Duplicate Rows Identified:", len(duplicates_df))
print("✔ Final Unique Fixtures:", len(clean_df))
print("\n🎯 Clean data saved successfully!")


📅 Fetching: 20251125

🔍 Total Raw Records: 105
♻ Duplicate Rows Identified: 76
✔ Final Unique Fixtures: 63

🎯 Clean data saved successfully!


In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import streamlit as st
from io import BytesIO

# ==============================
# CONFIG: ESPN API COMPETITIONS
# ==============================
COMPETITIONS = {
    "Men's NCAA Basketball": "basketball/mens-college-basketball",
    "Women's NCAA Basketball": "basketball/womens-college-basketball"
}

CONFERENCES = {
    "ACC": 2, "Big Ten": 7, "SEC": 8, "Big 12": 4, "Pac-12": 21, "Big East": 3, 
    "AAC": 62, "Mountain West": 46, "Atlantic 10": 3, "WCC": 26, "Sun Belt": 27,
    "MAAC": 5, "Horizon": 24, "Ivy League": 12, "Patriot League": 31, "America East": 1,
    "MEAC": 16, "SWAC": 23, "ASUN": 55, "Missouri Valley": 9, "CAA": 10,
    "Ohio Valley": 14, "Summit League": 44
}


# ===========================
# FETCH FIXTURES FUNCTION
# ===========================

def fetch_fixtures(start_date, end_date, gender_url):
    base_url = f"https://site.api.espn.com/apis/site/v2/sports/{gender_url}/scoreboard"
    all_games = []

    date_cursor = start_date
    while date_cursor <= end_date:
        date_str = date_cursor.strftime("%Y%m%d")
        
        for conference_name, group_id in CONFERENCES.items():
            url = f"{base_url}?dates={date_str}&groups={group_id}"
            resp = requests.get(url)

            if resp.status_code != 200:
                continue

            data = resp.json()

            for event in data.get("events", []):
                game = event["competitions"][0]

                teams = game["competitors"]
                status = game["status"]["type"]["description"]
                ko_time = event.get("date", None)

                # Convert KO Time
                if ko_time:
                    ko_time = datetime.fromisoformat(ko_time.replace('Z', '')).strftime("%I:%M %p")

                all_games.append({
                    "Date": date_cursor.strftime("%d-%m-%Y"),
                    "KO Time": ko_time,
                    "Competition": gender_url.split("/")[1],
                    "Conference": conference_name,
                    "Home Team": teams[0]["team"]["displayName"].upper(),
                    "Away Team": teams[1]["team"]["displayName"].upper(),
                    "Home Score": teams[0].get("score", None),
                    "Away Score": teams[1].get("score", None),
                    "Status": status
                })

        date_cursor += timedelta(days=1)

    df = pd.DataFrame(all_games)

    # Create Unique Key
    df["Game_Key"] = df["Date"] + "_" + df["Home Team"] + "_" + df["Away Team"]

    # Duplicate Flagging
    df["Is_Duplicate"] = df.duplicated(subset=["Game_Key"], keep=False)

    return df


# ===========================
# STREAMLIT UI
# ===========================

st.title("🏀 NCAA Full Season Fixture Extractor")

competition = st.selectbox("Select Competition:", list(COMPETITIONS.keys()))

col1, col2 = st.columns(2)
start_date = col1.date_input("Start Date", datetime.today())
end_date = col2.date_input("End Date", datetime.today())

if st.button("🔍 Fetch Fixtures"):
    st.info("Fetching data... Please wait.")

    gender_url = COMPETITIONS[competition]
    df = fetch_fixtures(datetime.combine(start_date, datetime.min.time()),
                        datetime.combine(end_date, datetime.min.time()),
                        gender_url)

    st.success(f"✔ Data Loaded — {len(df)} rows extracted")

    st.dataframe(df.head(50))

    # Duplicate subset
    dup_df = df[df["Is_Duplicate"] == True]

    # Create Excel with formatting
    output = BytesIO()
    
    with pd.ExcelWriter(output, engine="xlsxwriter") as writer:
        df.to_excel(writer, sheet_name="Full Fixtures", index=False)
        dup_df.to_excel(writer, sheet_name="Duplicate Fixtures", index=False)

        workbook = writer.book
        worksheet = writer.sheets["Full Fixtures"]

        highlight = workbook.add_format({"bg_color": "#FFDDDD", "font_color": "black"})

        for row_idx, is_dup in enumerate(df["Is_Duplicate"], start=1):
            if is_dup:
                worksheet.set_row(row_idx, cell_format=highlight)

    st.download_button(
        label="📥 Download Excel File",
        data=output.getvalue(),
        file_name=f"NCAA_Fixtures_{competition.replace(' ', '_')}_{start_date}_to_{end_date}.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )


2025-11-25 10:44:01.722 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44:04.665 
  command:

    streamlit run C:\Users\DINESH\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-25 10:44:04.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44:04.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44:04.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44:04.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44:04.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 10:44